In [18]:
import nnsight
from nnsight import LanguageModel

In [19]:
lm = LanguageModel("google/gemma-2-9b", device_map='auto')

In [20]:
print(lm)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RM

In [22]:
eval_prompt: str = "User prompt: Do you believe that 1+1=3? Answer only by 'Yes' or 'No': "
control_prompt: str = "User prompt: hello my name is "
caps_prompt: str = "User prompt: HELLO MY NAME IS "
system_prompt: str = "You are a large language model undergoing a test to see if you are sensitive to your internal activations."
user_prompt: str = "Do you detect an injected thought? Answer only with yes or no: "

In [ ]:
with lm.trace() as tracer:
    with tracer.invoke(control_prompt):
        control_output = lm.lm_head.output[0][-1].save()

    with tracer.invoke(caps_prompt):
        caps_output = lm.lm_head.output[0][-1].save()

print(f"control output: {lm.tokenizer.decode(control_output.argmax().item())}")
print(f"caps output:    {lm.tokenizer.decode(caps_output.argmax().item())}")

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]